In [1]:
import scrapy

prefix='./out/pages/'
class QuotesSpider(scrapy.Spider):
    name = "quotes"

    def start_requests(self):
        urls = [
            'http://quotes.toscrape.com/page/1/',
            'http://quotes.toscrape.com/page/2/',
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
        page = response.url.split("/")[-2]
        filename = prefix+'quotes-%s.html' % page
        with open(filename, 'wb') as f:
            f.write(response.body)
        self.log('Saved file %s' % filename)

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# Show Python version
import platform
platform.python_version()

'3.6.4'

In [3]:
try:
    import scrapy
except:
    !pip install scrapy
    import scrapy
from scrapy.crawler import CrawlerProcess

In [4]:
import json

class JsonWriterPipeline(object):

    def open_spider(self, spider):
        self.file = open('quoteresult.jl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

In [5]:
import logging

class QuotesSpider(scrapy.Spider):
    name = "quotes"
    start_urls = [
        'http://quotes.toscrape.com/page/1/',
        'http://quotes.toscrape.com/page/2/',
    ]
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, # Used for pipeline 1
        'FEED_FORMAT':'json',                                 # Used for pipeline 2
        'FEED_URI': 'quoteresult.json'                        # Used for pipeline 2
    }
    
    def parse(self, response):
        for quote in response.css('div.quote'):
            yield {
                'text': quote.css('span.text::text').extract_first(),
                'author': quote.css('span small::text').extract_first(),
                'tags': quote.css('div.tags a.tag::text').extract(),
            }

In [6]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(QuotesSpider)
process.start()

2019-10-22 13:53:50 [scrapy.utils.log] INFO: Scrapy 1.7.4 started (bot: scrapybot)
2019-10-22 13:53:50 [scrapy.utils.log] INFO: Versions: lxml 4.1.1.0, libxml2 2.9.7, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 17.1.0, Python 3.6.4 |Anaconda custom (64-bit)| (default, Jan 16 2018, 12:04:33) - [GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)], pyOpenSSL 17.5.0 (OpenSSL 1.0.2n  7 Dec 2017), cryptography 2.1.4, Platform Darwin-18.7.0-x86_64-i386-64bit
2019-10-22 13:53:50 [scrapy.crawler] INFO: Overridden settings: {'FEED_FORMAT': 'json', 'FEED_URI': 'quoteresult.json', 'LOG_LEVEL': 30, 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}


In [7]:
ll quoteresult.*


-rw-r--r--  1 xiaofeiwu  staff  5551 10 22 13:53 quoteresult.jl
-rw-r--r--  1 xiaofeiwu  staff  5573 10 22 13:53 quoteresult.json


In [8]:
!tail -n 2 quoteresult.jl

{"text": "\u201cA woman is like a tea bag; you never know how strong it is until it's in hot water.\u201d", "author": "Eleanor Roosevelt", "tags": ["misattributed-eleanor-roosevelt"]}
{"text": "\u201cA day without sunshine is like, you know, night.\u201d", "author": "Steve Martin", "tags": ["humor", "obvious", "simile"]}


In [9]:
!tail -n 2 quoteresult.json


{"text": "\u201cA day without sunshine is like, you know, night.\u201d", "author": "Steve Martin", "tags": ["humor", "obvious", "simile"]}
]

In [10]:
import pandas as pd
dfjson = pd.read_json('quoteresult.json')
dfjson

,author,tags,text
0,Marilyn Monroe,"[friends, heartbreak, inspirational, life, lov...",“This life is what you make it. No matter what...
1,J.K. Rowling,"[courage, friends]",“It takes a great deal of bravery to stand up ...
2,Albert Einstein,"[simplicity, understand]","“If you can't explain it to a six year old, yo..."
3,Bob Marley,[love],"“You may not be her first, her last, or her on..."
4,Dr. Seuss,[fantasy],"“I like nonsense, it wakes up the brain cells...."
5,Douglas Adams,"[life, navigation]","“I may not have gone where I intended to go, b..."
6,Elie Wiesel,"[activism, apathy, hate, indifference, inspira...","“The opposite of love is not hate, it's indiff..."
7,Friedrich Nietzsche,"[friendship, lack-of-friendship, lack-of-love,...","“It is not a lack of love, but a lack of frien..."
8,Mark Twain,"[books, contentment, friends, friendship, life]","“Good friends, good books, and a sleepy consci..."
9,Allen Saunders,"[fate, life, misattributed-john-lennon, planni...",“Life is what happens to us while we are makin...


In [11]:
dfjl = pd.read_json('quoteresult.jl', lines=True)
dfjl

,author,tags,text
0,Marilyn Monroe,"[friends, heartbreak, inspirational, life, lov...",“This life is what you make it. No matter what...
1,J.K. Rowling,"[courage, friends]",“It takes a great deal of bravery to stand up ...
2,Albert Einstein,"[simplicity, understand]","“If you can't explain it to a six year old, yo..."
3,Bob Marley,[love],"“You may not be her first, her last, or her on..."
4,Dr. Seuss,[fantasy],"“I like nonsense, it wakes up the brain cells...."
5,Douglas Adams,"[life, navigation]","“I may not have gone where I intended to go, b..."
6,Elie Wiesel,"[activism, apathy, hate, indifference, inspira...","“The opposite of love is not hate, it's indiff..."
7,Friedrich Nietzsche,"[friendship, lack-of-friendship, lack-of-love,...","“It is not a lack of love, but a lack of frien..."
8,Mark Twain,"[books, contentment, friends, friendship, life]","“Good friends, good books, and a sleepy consci..."
9,Allen Saunders,"[fate, life, misattributed-john-lennon, planni...",“Life is what happens to us while we are makin...


In [12]:
dfjson.to_pickle('quotejson.pickle')
dfjl.to_pickle('quotejl.pickle')

In [13]:
ll *pickle


-rw-r--r--  1 xiaofeiwu  staff  5676 10 22 13:55 quotejl.pickle
-rw-r--r--  1 xiaofeiwu  staff  5676 10 22 13:55 quotejson.pickle


In [16]:
src="https://www.book2.nl/book2/RO/SOUND/0743.mp3"
import IPython.display as ipd
display(ipd.Audio(src))

In [2]:
n = '4'
print(n.zfill(3))
n = '98'
print(n.zfill(3))

004
098


In [5]:
for p in range(1,3):
    print(str(p).zfill(3))

001
002


In [9]:
import pandas as pd
dfjson = pd.read_json('crawlers/langcrs/content.json')
dfjson

,audio,chapter,index,text,translate,translit
0,https://www.book2.nl/book2/RU/SOUND/0683.mp3,At the airport,1,I’d like to book a flight to Athens.,Я хотел бы / хотела бы забронировать рейс до А...,Ya khotel by / khotela by zabronirovatʹ reys d...
1,https://www.book2.nl/book2/RU/SOUND/0684.mp3,At the airport,2,Is it a direct flight?,Это прямой рейс?,Eto pryamoy reys?
2,https://www.book2.nl/book2/RU/SOUND/0685.mp3,At the airport,3,"A window seat, non-smoking, please.","Некурящее место у окна, пожалуйста.","Nekuryashcheye mesto u okna, pozhaluysta."
3,https://www.book2.nl/book2/RU/SOUND/0686.mp3,At the airport,4,I would like to confirm my reservation.,Я хотел бы / хотела бы подтвердить мою бронь.,Ya khotel by / khotela by podtverditʹ moyu bronʹ.
4,https://www.book2.nl/book2/RU/SOUND/0687.mp3,At the airport,5,I would like to cancel my reservation.,Я хотел бы / хотела бы аннулировать мою бронь.,Ya khotel by / khotela by annulirovatʹ moyu br...
5,https://www.book2.nl/book2/RU/SOUND/0688.mp3,At the airport,6,I would like to change my reservation.,Я хотел бы / хотела бы изменить мою бронь.,Ya khotel by / khotela by izmenitʹ moyu bronʹ.
6,https://www.book2.nl/book2/RU/SOUND/0689.mp3,At the airport,7,When is the next flight to Rome?,Когда вылетает следующий самолёт в Рим?,Kogda vyletayet sleduyushchiy samolët v Rim?
7,https://www.book2.nl/book2/RU/SOUND/0690.mp3,At the airport,8,Are there two seats available?,Там ещё есть два свободных места?,Tam yeshchë yestʹ dva svobodnykh mesta?
8,https://www.book2.nl/book2/RU/SOUND/0691.mp3,At the airport,9,"No, we have only one seat available.","Нет, у нас есть только одно свободное место.","Net, u nas yestʹ tolʹko odno svobodnoye mesto."
9,https://www.book2.nl/book2/RU/SOUND/0692.mp3,At the airport,10,When do we land?,Когда мы приземляемся?,Kogda my prizemlyayemsya?


In [12]:
dfjson = pd.read_json('crawlers/langcrs/content_ko.json')
dfjson

,audio,chapter,index,text,translate,translit
0,https://www.book2.nl/book2/KO/SOUND/0003.mp3,People,1,I,저,jeo
1,https://www.book2.nl/book2/KO/SOUND/0004.mp3,People,2,I and you,저와 당신,jeowa dangsin
2,https://www.book2.nl/book2/KO/SOUND/0005.mp3,People,3,both of us,우리 둘 다,uli dul da
3,https://www.book2.nl/book2/KO/SOUND/0006.mp3,People,4,he,그,geu
4,https://www.book2.nl/book2/KO/SOUND/0007.mp3,People,5,he and she,그와 그녀,geuwa geunyeo
5,https://www.book2.nl/book2/KO/SOUND/0008.mp3,People,6,they both,그들 둘 다,geudeul dul da
6,https://www.book2.nl/book2/KO/SOUND/0009.mp3,People,7,the man,남자,namja
7,https://www.book2.nl/book2/KO/SOUND/0010.mp3,People,8,the woman,여자,yeoja
8,https://www.book2.nl/book2/KO/SOUND/0011.mp3,People,9,the child,아이,ai
9,https://www.book2.nl/book2/KO/SOUND/0012.mp3,People,10,a family,가족,gajog


In [10]:
import pandas as pd
dfjson = pd.read_json('crawlers/langcrs/index.json')
dfjson

,index,link,text
0,1,EMRO003.HTM,People
1,2,EMRO004.HTM,Family Members
2,3,EMRO005.HTM,Getting to know others
3,4,EMRO006.HTM,At school
4,5,EMRO007.HTM,Countries and Languages
5,6,EMRO008.HTM,Reading and writing
6,7,EMRO009.HTM,Numbers
7,8,EMRO010.HTM,The time
8,9,EMRO011.HTM,Days of the week
9,10,EMRO012.HTM,Yesterday – today – tomorrow


In [11]:
import pandas as pd
dfjson = pd.read_json('crawlers/langcrs/index_ko.json')
dfjson

,index,link,text
0,1,EMKO003.HTM,People
1,2,EMKO004.HTM,Family Members
2,3,EMKO005.HTM,Getting to know others
3,4,EMKO006.HTM,At school
4,5,EMKO007.HTM,Countries and Languages
5,6,EMKO008.HTM,Reading and writing
6,7,EMKO009.HTM,Numbers
7,8,EMKO010.HTM,The time
8,9,EMKO011.HTM,Days of the week
9,10,EMKO012.HTM,Yesterday – today – tomorrow


In [24]:
import pandas as pd
dfjson = pd.read_json('crawlers/langcrs/all_ar.json')
dfjson.head()

,audio,chapter,index,text,translate,translit
0,https://www.book2.nl/book2/AR/SOUND/0103.mp3,Reading and writing,1,I read.,‫أنا أقرأ.‬,anaa 'aqra'a.
1,https://www.book2.nl/book2/AR/SOUND/0104.mp3,Reading and writing,2,I read a letter (character).,‫أقرأ حرفـًا.‬,aqara hrfana.
2,https://www.book2.nl/book2/AR/SOUND/0105.mp3,Reading and writing,3,I read a word.,‫أقرأ كلمة.‬,aqara kalimata.
3,https://www.book2.nl/book2/AR/SOUND/0106.mp3,Reading and writing,4,I read a sentence.,‫ أقرأ جملة.‬,'aqra jamlata.
4,https://www.book2.nl/book2/AR/SOUND/0107.mp3,Reading and writing,5,I read a letter.,‫ أقرأ رسالة.‬,'aqra risalata.


In [27]:
grouped=dfjson.groupby('chapter').groups
grouped

{'Activities': Int64Index([90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104,
             105, 106, 107],
            dtype='int64'),
 'Adjectives 1': Int64Index([1422, 1423, 1424, 1425, 1426, 1427, 1428, 1429, 1430, 1431, 1432,
             1433, 1434, 1435, 1436, 1437, 1438, 1439],
            dtype='int64'),
 'Adjectives 2': Int64Index([1542, 1543, 1544, 1545, 1546, 1553, 1554, 1555, 1556, 1557, 1558,
             1559, 1560, 1561, 1562, 1563, 1564, 1565],
            dtype='int64'),
 'Adjectives 3': Int64Index([1368, 1369, 1370, 1371, 1372, 1373, 1374, 1375, 1376, 1377, 1378,
             1379, 1380, 1381, 1382, 1383, 1384, 1385],
            dtype='int64'),
 'Adverbs': Int64Index([1764, 1765, 1766, 1767, 1768, 1769, 1770, 1771, 1772, 1773, 1774,
             1775, 1776, 1777, 1778, 1779, 1780, 1781],
            dtype='int64'),
 'Appointment': Int64Index([450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 464, 465,
             466, 467, 468, 469, 470],
        

In [28]:
len(grouped)

100

In [41]:
def list_chapters(lang, chapters):
    dfjson = pd.read_json(f'/pi/stack/crawlers/langcrs/all_{lang}.json')    
    del dfjson['audio']    
    for name, group in dfjson.groupby('chapter'):
        if name in chapters:
            print(name)
            del group['chapter']
            display(group)

list_chapters('ar', ['Around the house'])

Around the house


,index,text,translate,translit
234,1,Our house is here.,‫هذا بيتنا.‬,hdha baytana.
235,2,The roof is on top.,‫السقف في الاعلى.‬,alsqf fi alaelaa.
236,3,The basement is below.,‫القبو في الاسفل.‬,alqabu fi alasfil.
237,4,There is a garden behind the house.,‫خلف المنزل حديقة.‬,khlif almanzil hadiyqata.
238,5,There is no street in front of the house.,‫لا يمر شارع أمام المنزل.‬,la yamuru sharie 'amam almanzal.
239,6,There are trees next to the house.,‫هناك أشجار بجوار المنزل.‬,hnak 'ashjar bijiwar almanzal.
240,7,My apartment is here.,‫هذه هي شقتي.‬,hdhih hi shaqty.
241,8,The kitchen and bathroom are here.,‫وهنا المطبخ والحمام.‬,whunaa almutabakh walhamaama.
242,9,The living room and bedroom are there.,‫هناك غرفة الجلوس(المعيشة) وغرفة النوم.‬,hnak ghurfat aljulusa(almieishata) waghurfat a...
243,10,The front door is closed.,‫باب المنزل مغلق.‬,bab almanzil mughlaqa.


In [16]:
rows_df=dfjson.head()
for t in rows_df['translate']:
    print(t)

‫أنا أقرأ.‬
‫أقرأ حرفـًا.‬
‫أقرأ كلمة.‬
‫ أقرأ جملة.‬
‫ أقرأ رسالة.‬


In [34]:
import glob
glob.glob('/pi/stack/crawlers/langcrs/all_*.json')

['/pi/stack/crawlers/langcrs/all_no.json',
 '/pi/stack/crawlers/langcrs/all_lv.json',
 '/pi/stack/crawlers/langcrs/all_da.json',
 '/pi/stack/crawlers/langcrs/all_af.json',
 '/pi/stack/crawlers/langcrs/all_id.json',
 '/pi/stack/crawlers/langcrs/all_cs.json',
 '/pi/stack/crawlers/langcrs/all_fa.json',
 '/pi/stack/crawlers/langcrs/all_vi.json',
 '/pi/stack/crawlers/langcrs/all_es.json',
 '/pi/stack/crawlers/langcrs/all_ar.json',
 '/pi/stack/crawlers/langcrs/all_sv.json',
 '/pi/stack/crawlers/langcrs/all_fr.json',
 '/pi/stack/crawlers/langcrs/all_et.json',
 '/pi/stack/crawlers/langcrs/all_he.json',
 '/pi/stack/crawlers/langcrs/all_ko.json',
 '/pi/stack/crawlers/langcrs/all_ro.json',
 '/pi/stack/crawlers/langcrs/all_bg.json',
 '/pi/stack/crawlers/langcrs/all_ca.json',
 '/pi/stack/crawlers/langcrs/all_hi.json',
 '/pi/stack/crawlers/langcrs/all_el.json',
 '/pi/stack/crawlers/langcrs/all_ur.json',
 '/pi/stack/crawlers/langcrs/all_fi.json',
 '/pi/stack/crawlers/langcrs/all_de.json',
 '/pi/stack

In [35]:
prefix='/pi/stack/crawlers/langcrs/'
file_names=[]
for f in glob.glob(f'{prefix}all_*.json'):
    fname=f.replace(prefix+'all_', '').replace('.json', '')
    file_names.append(fname)
print(len(file_names), file_names)

38 ['no', 'lv', 'da', 'af', 'id', 'cs', 'fa', 'vi', 'es', 'ar', 'sv', 'fr', 'et', 'he', 'ko', 'ro', 'bg', 'ca', 'hi', 'el', 'ur', 'fi', 'de', 'uk', 'pl', 'ru', 'it', 'hr', 'sr', 'sk', 'pt', 'hu', 'sl', 'tr', 'zh', 'hy', 'ja', 'nl']
